In [152]:
# %pip install statsmodels
# %pip install pandas
# %pip install matplotlib

In [153]:
VOLZA_FILE_PATH = '../../../volza/magnesium/magnesium.csv'

# The number of countries to analyze. 10 is the default, but you can change it to whatever you want
# We will only analyze the top 10 countries by the number of samples
NUMBER_OF_COUNTRIES_TO_ANALYZE = 5

# Column names just so we don't have to constantly type them
VALUE_COLUMN='Value'
COUNTRY_OF_ORIGIN_COLUMN='Country of Origin'
COUNTRY_OF_DESTINATION_COLUMN='Country of Destination'

TIME_METRIC = '7D'

In [154]:
import pandas
data = pandas.read_csv(VOLZA_FILE_PATH)
data['Date'] = pandas.to_datetime(data['Date'])
data.sort_values('Date', inplace=True)
data.set_index('Date', inplace=True)

In [155]:
import warnings
warnings.filterwarnings('ignore')

# **Stationarity**

### **ADF Test** (Stationarity)

In [156]:
from statsmodels.tsa.stattools import adfuller

def adf_test(df) -> dict:
    result = adfuller(df.values)

    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value), end=' \t')
        print('\tIs stationary: %s' % ('No', 'Yes')[1 if result[0] < result[4][key] else 0])
        # to_return['Critical value (%s)' % key] = value

    return {
        'ADF': result[0],
        'ADF p-value': result[1],
    }

### **KPSS Test** (Trend Stationarity)

In [157]:
from statsmodels.tsa.stattools import kpss
def kpss_test(df) -> dict:
  """Tests for stationarity, null hypothesis is that the series is stationary"""
  statistic, p_value, n_lags, critical_values = kpss(df.values)
  print(f'KPSS Statistic: {statistic}')
  print(f'p-value: {p_value}')
  print(f'num lags: {n_lags}')
  print('Critial Values:')
  for key, value in critical_values.items():
    print('\t%s: %.3f' % (key, value), end=' \t')
    print('\tIs stationary: %s' % ('No', 'Yes')[1 if p_value < float(key[:-1]) / 100 else 0])
  return {
    'KPSS': statistic,
    'KPSS p-value': p_value,
  }

# **Autocorrelation / Partial Autocorrelation**

In [158]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

def autocorrelation_plots(df, lag):
  fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
  plot_acf(df, lags=lag, ax=axes[0])
  plot_pacf(df, lags=lag, ax=axes[1], method='ywm')
  plt.show()


### **Pearson and Spearman Correlation**

In [159]:
from scipy.stats import pearsonr, spearmanr

def spearman_correlation(original, target):
  print("Spearman Correlation")
  print(spearmanr(original, target))

def pearson_correlation(original, target):
  print("Pearson Correlation")
  print(pearsonr(original, target))

# **Granger Causality Test**

In [160]:
from statsmodels.tsa.stattools import grangercausalitytests
import pandas
import numpy

# Granger causality test which accounts for unmatched lengths of the two series
def grangercausality_test(combined, maxlag):
  grangercausalitytests(combined, maxlag=maxlag, verbose=True)

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False, maxlag=10):    
   
    df = pandas.DataFrame(numpy.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = numpy.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [161]:
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.api import VAR

def perform_analysis(dataframe, countries, granger_file_name, statistics_file_name):
    country_results = []
    for country in countries:
        print("=====================================")
        print(country)
        print("=====================================")
        # Get sum of Value each day
        data_for_country = dataframe.get_group(country)
        data_for_country = data_for_country[VALUE_COLUMN].resample(TIME_METRIC).sum()
        
        # Stationarity tests
        try:
            adf_test_result = adf_test(data_for_country)
            kpss_test_result = kpss_test(data_for_country)
            
        except: 
            adf_test_result = {
                'ADF': "Not enough data",
                'ADF p-value': "Not enough data",
            }
            kpss_test_result = {
                'KPSS': "Not enough data",
                'KPSS p-value': "Not enough data",
            }
        country_result = { 'Country': country, **adf_test_result, **kpss_test_result}
        country_results.append(country_result)
        # Autocorrelation plots
        # autocorrelation_plots(data_for_country, lag=10)

        print()
        print()
        print()
        print()
        print()
    
    
    country_results = pandas.DataFrame(country_results).to_csv(statistics_file_name)
    
    countries_combined = pandas.DataFrame({country: dataframe.get_group(country)[VALUE_COLUMN].resample(TIME_METRIC).sum()  for country in countries}).fillna(0)
    granger_results = grangers_causation_matrix(countries_combined, countries, maxlag=10)
    granger_results.to_csv(granger_file_name)
    countries_combined_transformed = countries_combined.diff().dropna()
    model = VAR(countries_combined_transformed)

    maxlags = 0
    minaic = 10000000
    for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
        result = model.fit(i)
        if result.aic < minaic:
            minaic = result.aic
            maxlags = i
    results = model.fit(maxlags=maxlags, ic='aic')
    out = durbin_watson(results.resid)

    for col, val in zip(countries_combined.columns, out):
        print(col, ':', round(val, 2))       

print("===============================================================================================================")
print("Country of Origin")
print("===============================================================================================================")
COUNTRY_OF_ORIGINS_TO_ANALYZE = data.groupby(COUNTRY_OF_ORIGIN_COLUMN).size().sort_values(ascending=False).head(NUMBER_OF_COUNTRIES_TO_ANALYZE).index.values.tolist()
data_grouped_by_country_of_origin = data.groupby(COUNTRY_OF_ORIGIN_COLUMN)
perform_analysis(data_grouped_by_country_of_origin, COUNTRY_OF_ORIGINS_TO_ANALYZE, "Granger Causality Results Grouped by Country of Origin.csv", "Statistics Grouped by Country of Origin.csv")

print("===============================================================================================================")
print("Country of Destination")
print("===============================================================================================================")
COUNTRY_OF_DESTINATIONS_TO_ANALYZE = data.groupby(COUNTRY_OF_DESTINATION_COLUMN).size().sort_values(ascending=False).head(NUMBER_OF_COUNTRIES_TO_ANALYZE).index.values.tolist()
data_grouped_by_country_of_destination = data.groupby(COUNTRY_OF_DESTINATION_COLUMN)
perform_analysis(data_grouped_by_country_of_destination, COUNTRY_OF_ORIGINS_TO_ANALYZE, "Granger Causality Results Grouped by Country of Destination.csv", "Statistics Grouped by Country of Destination.csv")

Country of Origin
China
ADF Statistics: -2.089025
p-value: 0.248951
Critical values:
	1%: -3.475 		Is stationary: No
	5%: -2.881 		Is stationary: No
	10%: -2.577 		Is stationary: No
KPSS Statistic: 0.6834146937415495
p-value: 0.015053209659859134
num lags: 7
Critial Values:
	10%: 0.347 		Is stationary: Yes
	5%: 0.463 		Is stationary: Yes
	2.5%: 0.574 		Is stationary: Yes
	1%: 0.739 		Is stationary: No





Netherlands
ADF Statistics: -1.592620
p-value: 0.487300
Critical values:
	1%: -3.478 		Is stationary: No
	5%: -2.882 		Is stationary: No
	10%: -2.578 		Is stationary: No
KPSS Statistic: 0.3101298637677548
p-value: 0.1
num lags: 57
Critial Values:
	10%: 0.347 		Is stationary: No
	5%: 0.463 		Is stationary: No
	2.5%: 0.574 		Is stationary: No
	1%: 0.739 		Is stationary: No





Germany
ADF Statistics: -1.846325
p-value: 0.357717
Critical values:
	1%: -3.478 		Is stationary: No
	5%: -2.882 		Is stationary: No
	10%: -2.578 		Is stationary: No
KPSS Statistic: 0.37854792971151396
p-value: 